In [ ]:
import mysql.connector
from datetime import datetime
import tkinter as tk
from tkinter import messagebox

# Connect to the MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="parking"
)

cursor = db.cursor()

# Function to register a vehicle
def register_vehicle():
    plate_number = plate_number_entry.get()
    vehicle_type = vehicle_type_entry.get()
   
    # Check if the vehicle is already registered
    check_query = "SELECT * FROM vehicles WHERE plate_number = %s"
    data = (plate_number,)
    cursor.execute(check_query, data)
    result = cursor.fetchone()

    if result:
        messagebox.showinfo("Info", "Vehicle is already registered.")
    else:
        entry_time = datetime.now()
        insert_query = "INSERT INTO vehicles (plate_number, vehicle_type, entry_time) VALUES (%s, %s, %s)"
        data = (plate_number, vehicle_type, entry_time)
        cursor.execute(insert_query, data)
        db.commit()
        messagebox.showinfo("Success", "Vehicle registered successfully.")

# Function to allocate parking and calculate fee
def allocate_parking():
    plate_number = plate_number_entry.get()
   
    # Check if the vehicle is registered
    check_query = "SELECT * FROM vehicles WHERE plate_number = %s"
    data = (plate_number,)
    cursor.execute(check_query, data)
    vehicle = cursor.fetchone()

    if not vehicle:
        messagebox.showinfo("Info", "Vehicle is not registered.")
        return

    exit_time = datetime.now()
    entry_time = vehicle[3]  # Entry time from the database
    duration = (exit_time - entry_time).seconds // 3600  # Calculate duration in hours
    fee = calculate_fee(duration, vehicle[2])  # Calculate parking fee based on vehicle type

    messagebox.showinfo("Parking Fee", f"Vehicle parked for {duration} hours. Parking fee: ${fee:.2f}")

    # Remove the vehicle from the database
    delete_query = "DELETE FROM vehicles WHERE plate_number = %s"
    cursor.execute(delete_query, data)
    db.commit()

# Function to calculate parking fee based on vehicle type and duration
def calculate_fee(duration, vehicle_type):
    # You can define your own fee calculation logic based on vehicle type and duration
    # For simplicity, this example uses a fixed fee per hour
    if vehicle_type == "car":
        return duration * 5  # $5 per hour for cars
    elif vehicle_type == "motorcycle":
        return duration * 2  # $2 per hour for motorcycles
    else:
        return 0

# Create the main window
root = tk.Tk()
root.title("Parking Management System")

# Create labels and entry widgets for vehicle registration and allocation
tk.Label(root, text="Vehicle Plate Number:").pack()
plate_number_entry = tk.Entry(root)
plate_number_entry.pack()

tk.Label(root, text="Vehicle Type (car/motorcycle):").pack()
vehicle_type_entry = tk.Entry(root)
vehicle_type_entry.pack()

# Create buttons for vehicle registration and allocation
register_button = tk.Button(root, text="Register Vehicle", command=register_vehicle)
register_button.pack()

allocate_button = tk.Button(root, text="Allocate Parking", command=allocate_parking)
allocate_button.pack()

# Main loop for the Tkinter GUI
root.mainloop()

# Close the database connection when done
db.close()